In [ ]:
import random, time, statistics
import numpy as np
import pandas as pd
import multiprocessing as mp
import numba as nb

# Topics

0. Python Functions
1. Multiprocessing
2. NumPy
3. Pandas
4. Cython
5. Numba

## Python Functions


```
def function_name(optional_parameters):
    function_code
    return optional_return_variables
```

Functions encapsulate a set of instructions that can be reused. The following tips for faster Python all benefit from writing code that is well functionalized. Further, using functions are generally a good coding practice as they allow for the blocks of code to be written once and then reused many times. Then if the code needs to be updated it only needs to be updated once and not many times.

### Example: Functions estimating π via a Monte-Carlo algorithm

In [ ]:
def monte_carlo_pi(points):
    s = 0
    for _ in range(points):
        x = random.random()**2
        y = random.random()**2
        if x + y < 1:
            s += 1
    return 4. * float(s) / float(points)

def sample_points(p):
    s = []
    for i in range(p):
        s.append(monte_carlo_pi(10**p))
    return s

def print_sample(s):
    for p in s:
        print(p)

In [ ]:
print_sample(sample_points(7))

### Task: Create functions estimating π via a Gauss–Legendre algorithm

Initial:

$a_0 = 1$

$b_0 = \frac{1}{\sqrt{2}}$

$t_0 = \frac{1}{4}$

$p_0 = 1$

Loop until $a_{n}$ and $b_{n}$ difference meets threashold:

$a_{n+1} = \frac{a_{n}+b_{n}}{2}$

$b_{n+1} = \sqrt{a_{n}b_{n}}$

$t_{n+1} = t_{n}-p_{n}\sqrt{a_{n}-a_{n+1}}$

$p_{n+1} = 2p_{n}$

$\pi \approx \frac{(a_{n+1}+b_{n+1})^2}{4t_{n+1}}$

## [NumPy](https://numpy.org)

NumPy is the fundamental package for scientific computing with Python. It contains among other things:

* a powerful N-dimensional array object
* sophisticated (broadcasting) functions
* tools for integrating C/C++ and Fortran code
* useful linear algebra, Fourier transform, and random number capabilities

Besides its obvious scientific uses, NumPy can also be used as an efficient multi-dimensional container of generic data. Arbitrary data-types can be defined. This allows NumPy to seamlessly and speedily integrate with a wide variety of databases.

Many NumPy functions are compiled libraries using fast BLAS and LAPACK implementations. These functions and functions built on top of NumPy functions will likely run faster than pure Python implementations.

### Example: Python Matrix-Matrix Multiplication

In [ ]:
def initialize_matrix(m, n, fill=0):
    matrix = []
    for i in range(m):
        row = []
        for j in range(n):
            if fill == 0:
                f = 0
            else:
                f = random.random()
            row.append(f)
        matrix.append(row)
    return matrix

def gemm(matrix_a, matrix_b):
    m = len(matrix_a[:][0])
    n = len(matrix_b[:][0])
    p = len(matrix_a[0][:])
    matrix_c = initialize_matrix(m, p)
    for i in range(m):
        for j in range(p):
            for k in range(n):
                matrix_c[i][j] += matrix_a[i][k]*matrix_b[k][j]
    return matrix_c

In [ ]:
matrix_a = [[1,2],[3,4]]
matrix_b = [[1,2],[3,4]]
matrix_c = gemm(matrix_a, matrix_b)
print(matrix_c)

In [ ]:
matrix_a = np.array(matrix_a)
matrix_b = np.array(matrix_b)
matrix_c = np.matmul(matrix_a, matrix_b)
print(matrix_c)

### Task: Create a function that compares the speed of Python and NumPy implementations of matrix-matrix multiplication

Use random square matrices of size 500, 1000, and 1500. Time the intialization and multiplication collectively using `time.time()`. Hints: [`rand()`](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.random.rand.html) and [`matmul()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matmul.html).